В цьому домашньому завданні ми проведемо додаткові експерименти для рішення задачі бінарної класифікації і створимо ваш новий submission на змагання на Kaggle.

-----------


**Завдання 0**. Завантажте дані `train.csv`, `test.csv`, `sample_submission.csv` зі змагання на Kaggle - шукайте посилання в уроці [Запрошення до участі у Kaggle-змаганні.](https://data-loves.kwiga.com/courses/machine-learning-dlia-liudei/domashnie-zavdannia-zmagannia-z-kaggle)  Для завантаження потрібно долучитись до змагання (натиснути кнопку "Join").


**Завдання 1**. **Збираємо весь код з попереднього ДЗ в одному місці.** В лекційному ноутбуці `Логістична регресія з ScikitLearn. Повна ML задача.ipynb` ми познайомились з поняттям пайплайнів, а також я показала, як компактно виглядає рішення МЛ задачі, якщо ми зберемо весь код разом.

Оскільки ми далі будемо робити експерименти, які включають ті самі етапи попередньої обробки, але інше моделювання - буде зручно мати весь код компактно і під рукою. Тому зараз ми займемось збором коду до купи :) Після цього завдання для подальших експериментів ви можете перенести частини розвʼязку взагалі в окремий `.py` файл, аби було зручно імпортувати функції.

Зі свого рішення в попередньому домашньому завданні (`Логістична регресія з scikit learn.ipynb`) зберіть усі кроки розвʼязку задачі разом з використанням `sklearn.Pipeline` за прикладом з лекції.

Ваш код нижче має містити
1. Читання даних з файлу (поза пайплайном).
2. Розбиття на тренувальний і валідаційний набори, де валідаційний містить 20% даних (поза пайплайном).
3. Виділення категоріальних і числових колонок (поза пайплайном).
4. Підготовку категоріальних і числових колонок (частина пайплайну). В прикладі в лекції ми оформлювали обробку числових і категоріальних колонок в окремі трансформери `numeric_transformer`, `categorical_cols`. Рекоемндую зробити саме так, так потім зручніше вносити зміни :)
5. Тренування лог регресії (частина пайплайну).
6. Запуск пайплайну на тренування на трен. даних (поза пайплайном).
7. Запуск пайплайну на передбачення на трен і вал. даних і вимір метрик якості ROC-AUC + вивдення Confusion Matrix (поза пайплайном).
8. Збереження моделі в формат joblib (поза пайплайном).

Ви це все вже зробили в попереднтьому ДЗ! Тож, тут просто заадча все зібрати разом.

Нижче я додала підказки, що покроково ви маєте зробити. Якщо ви почуваєтесь впевнено, можете видалити ці підказки і реалізувати все самостійно, або ж - просто заповнити пропуски.

Завдання оцінюється в 10 балів. Головний результат - аби код в фіналі був робочий. Бо за не робочий нам гроші ніхто не заплатить :)

In [15]:
# 0
import opendatasets as od
import os
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import f1_score, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import operator
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import root_mean_squared_error
from sklearn.linear_model import Ridge, Lasso, ElasticNet

In [ ]:
# Завантажуємо датасет
dataset_url= 'https://www.kaggle.com/competitions/bank-customer-churn-prediction-dlu-course-c-3/data'
od.download(dataset_url)

In [ ]:
# Зчитуємо із файлів, поділяємо датасет на тренувальний і валідаційний набори
data_dir = 'bank-customer-churn-prediction-dlu-course-c-3'
raw_train = pd.read_csv(data_dir + '/train.csv', index_col=0)
raw_test = pd.read_csv(data_dir + '/test.csv')

train_df, val_df = train_test_split(raw_train, test_size=0.20, random_state=42)

# Визначаємо список ознак і цільової змінної, виділяємо цільову із датасета
input_cols = list(train_df.columns)[1:-1]
target_col = 'Exited'
train_inputs, train_targets = train_df[input_cols], train_df[target_col]
val_inputs, val_targets = val_df[input_cols], val_df[target_col]

# Виявляємо числові і категоріальні колонки
numeric_cols_raw = train_inputs.select_dtypes(include=np.number).columns.tolist()
categorical_cols_raw = train_inputs.select_dtypes(include='object').columns.tolist()
non_use_col = ['Surname'] # Визначаємо ознаки, які не будемо використовувати
numeric_cols = [item for item in numeric_cols_raw if item not in non_use_col] # Визначаємо чисельні ознаки без невикористовуємих
categorical_cols = [item for item in categorical_cols_raw if item not in non_use_col] # Визначаємо категоріальні ознаки без невикористовуємих

'''# Обробляємо деякі колонки
input_cols.append('Balance_codes')
train_inputs['Balance_codes'] = train_inputs['Balance'].map(lambda x: 0 if x==0 else 1) # Додамо бінарну числову ознаку - є залишок грошей на балансі чи ні.
val_inputs['Balance_codes'] = val_inputs['Balance'].map(lambda x: 0 if x==0 else 1)
'''

# Створюємо трансформери для числових і категоріальних колонок
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler())
])

numeric_transformer_poly2 = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler()),
    ('polynomial_features', PolynomialFeatures(degree=2))
])

numeric_transformer_poly4 = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler()),
    ('polynomial_features', PolynomialFeatures(degree=4))
])

numeric_transformer_poly5 = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler()),
    ('polynomial_features', PolynomialFeatures(degree=5))
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

# Комбінуємо трансформери для різних типів колонок в один препроцесор
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

preprocessor_poly2 = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_poly2, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

preprocessor_poly4 = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_poly4, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

preprocessor_poly5 = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_poly5, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Стоврюємо пайплайн, який спочатку запускає препроцесинг, потім тренуєм модель
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(solver='liblinear', class_weight={0: 1, 1: 4})) # Додаємо ваги для класів
])

model_pipeline_poly2 = Pipeline(steps=[
    ('preprocessor', preprocessor_poly2),
    ('classifier', LogisticRegression(solver='liblinear', class_weight={0: 1, 1: 4})) # Додаємо ваги для класів
])

model_pipeline_poly4 = Pipeline(steps=[
    ('preprocessor', preprocessor_poly4),
    ('classifier', LogisticRegression(solver='liblinear', class_weight={0: 1, 1: 4})) # Додаємо ваги для класів
])

model_pipeline_poly5 = Pipeline(steps=[
    ('preprocessor', preprocessor_poly5),
    ('classifier', LogisticRegression(solver='liblinear', class_weight={0: 1, 1: 4})) # Додаємо ваги для класів
])

# Тренуємо пайплайн
model_pipeline.fit(train_inputs, train_targets)
model_pipeline_poly2.fit(train_inputs, train_targets)
model_pipeline_poly4.fit(train_inputs, train_targets)
model_pipeline_poly5.fit(train_inputs, train_targets)

# Функція, щоб передбачати і рахувати метрики
def predict_and_plot(model_pipeline, inputs, targets, name=''):
    preds = model_pipeline.predict(inputs)
    fpr, tpr, _ = roc_curve(targets, preds)
    roc_auc = auc(fpr, tpr)
    print(f"Area under ROC score on {name} dataset: {roc_auc:.2f}%")
    confusion_matrix_ = confusion_matrix(targets, preds, normalize='true')
    plt.figure()
    sns.heatmap(confusion_matrix_, annot=True, cmap='Blues')
    plt.xlabel('Prediction')
    plt.ylabel('Target')
    plt.title('{} Confusion Matrix'.format(name))
    plt.show()
    return preds

# Оцінюємо модель на трен і вал даних
train_preds = predict_and_plot(model_pipeline, train_inputs, train_targets, 'Train')
val_preds = predict_and_plot(model_pipeline, val_inputs, val_targets, 'Validation')

'''# Оцінюємо модель з поліноміальними фічами - в п.3
train_preds = predict_and_plot(model_pipeline_poly, train_inputs, train_targets, 'Train')
val_preds = predict_and_plot(model_pipeline_poly, val_inputs, val_targets, 'Validation')
'''
# Зберігаємо модель для подальшого використання
joblib.dump(model_pipeline, 'log_reg.joblib')



**Завдання 2**. Такс, у нас з вами є вже готовий пайплайн. Давайте проведемо нові експерименти.

  Додайте в попередню обробку числових колонок генерацію polinomal features до степені 2 включно. Для цього створіть новий препроцесор і створіть новий пайплайн.

  Запустіть пайплайн на тренування і виведіть метрики для тренувального і валідаційного набору. Напишіть, як вам модель? Чи спостерігається в цій моделі overfit чи underfit? Чи ця модель добре генералізує?

In [ ]:
# Оцінюємо модель з поліноміальними фічами
train_preds = predict_and_plot(model_pipeline_poly2, train_inputs, train_targets, 'Train')
val_preds = predict_and_plot(model_pipeline_poly2, val_inputs, val_targets, 'Validation')



Модель стала краще - збільшився AUC, при цьому метрики тренувального та валідаційного наборів як і раніше, схожі. Тому можемо зробити висновок, що немає ні оверфіту, ні андерфіту. Модель генералізує гарно.

**Завдання 3**. Тепер давайте створимо ще новий пайплайн, тільки тепер поліноміальні ознаки згенеруємо до степені 4. Зробіть висновок про якість моделі. Якщо вам подобається резульат якоїсь з моделей в цьому ДЗ - рекомендую зробити submission в змаганні.

In [ ]:
# degree_poly = 4
# Оцінюємо модель з поліноміальними фічами
train_preds = predict_and_plot(model_pipeline_poly4, train_inputs, train_targets, 'Train')
val_preds = predict_and_plot(model_pipeline_poly4, val_inputs, val_targets, 'Validation')

У степеню 4 покращилась метрика AUC тренувального набору - 0,86, аде для валідаційного залишилась як і раніше. Можно говорити про overfit.

In [ ]:
# Спробуємо поліноміалі фічі 2-го порядка на змаганні
sample_submission_csv = data_dir + '/sample_submission.csv'
sample_submission = pd.read_csv(sample_submission_csv)
sample_submission.info()

In [ ]:
raw_test['Exited'] = model_pipeline_poly2.predict(raw_test)

In [ ]:
sample_submission['Exited'] = sample_submission['id'].map(raw_test.set_index('id')['Exited'])
sample_submission.head(5)

In [ ]:
sample_submission.to_csv('D:/DS_UA/2_2 Logistic regression/HW/submission_log_reg.csv', index=False)

**Завдання 4. Перенавчання і регуляризація**.

  Скачайте набір даних `regression_data.csv`. Звичайте набір даних з `regression_data.csv`, розбийте на train і test (в тест 20%) і натренуйте модель лінійної регресії з масштабуванням числових ознак і поліноміальними ознаками до степені **5 включно**.

  Виміряйте якість прогностичної моделі і зробіть висновок, чи модель хороша, чи вона добре генералізує?


In [5]:
raw_regression = pd.read_csv('regression_data.csv')
raw_regression.head(2)

,feature_1,feature_2,feature_3,feature_4,feature_5,target
0,-0.190339,-1.382800,-0.875618,0.538910,-1.037246,28.938854
1,-0.321386,-0.563725,0.412931,-0.147057,-0.825497,-7.664581


In [6]:
X = raw_regression.drop(['target'], axis=1)
y = raw_regression['target']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
def train_lin_vs_poly_reg_and_plot(X_train, X_val, y_train, y_val, degree):
  # Звичайна лінійна регресія
  lin_reg = LinearRegression()
  lin_reg.fit(X_train, y_train)
  y_pred_train_lin = lin_reg.predict(X_train)
  y_pred_val_lin = lin_reg.predict(X_val)

  # Поліноміальна регресія
  poly_features = PolynomialFeatures(degree=degree)
  X_train_poly = poly_features.fit_transform(X_train)
  X_val_poly = poly_features.transform(X_val)

  print(f'В оригіналній матриці Х {X.shape[1]} ознак.')
  print(f'В матриці Х з поліноміальними ознаками {X_train_poly.shape[1]} ознак.\n')

  poly_reg = LinearRegression()
  poly_reg.fit(X_train_poly, y_train)
  y_pred_train_poly = poly_reg.predict(X_train_poly)
  y_pred_val_poly = poly_reg.predict(X_val_poly)

  # Оцінка моделей

  rmse_lin_train = root_mean_squared_error(y_pred_train_lin, y_train)
  rmse_poly_train = root_mean_squared_error(y_pred_train_poly, y_train)

  rmse_lin_val = root_mean_squared_error(y_pred_val_lin, y_val)
  rmse_poly_val = root_mean_squared_error(y_pred_val_poly, y_val)


  print(f"Train RMSE for Linear Regression: {np.sqrt(rmse_lin_train):.3f}")
  print(f"Validation RMSE for Linear Regression: {np.sqrt(rmse_lin_val):.3f}\n")

  print(f"Train RMSE for Polynomial Regression (degree {degree}): {np.sqrt(rmse_poly_train):.3f}")
  print(f"Validation RMSE for Polynomial Regression (degree {degree}): {np.sqrt(rmse_poly_val):.3f}\n")

In [19]:
for degree in range(2,6):
    print('degree=', degree)
    train_lin_vs_poly_reg_and_plot(X_train, X_val, y_train, y_val, degree)

degree= 2
В оригіналній матриці Х 5 ознак.
В матриці Х з поліноміальними ознаками 21 ознак.

Train RMSE for Linear Regression: 1.033
Validation RMSE for Linear Regression: 0.940

Train RMSE for Polynomial Regression (degree 2): 1.008
Validation RMSE for Polynomial Regression (degree 2): 1.007

degree= 3
В оригіналній матриці Х 5 ознак.
В матриці Х з поліноміальними ознаками 56 ознак.

Train RMSE for Linear Regression: 1.033
Validation RMSE for Linear Regression: 0.940

Train RMSE for Polynomial Regression (degree 3): 0.894
Validation RMSE for Polynomial Regression (degree 3): 1.384

degree= 4
В оригіналній матриці Х 5 ознак.
В матриці Х з поліноміальними ознаками 126 ознак.

Train RMSE for Linear Regression: 1.033
Validation RMSE for Linear Regression: 0.940

Train RMSE for Polynomial Regression (degree 4): 0.000
Validation RMSE for Polynomial Regression (degree 4): 4.390

degree= 5
В оригіналній матриці Х 5 ознак.
В матриці Х з поліноміальними ознаками 252 ознак.

Train RMSE for Linea

В оригінальній моделі спостерігається незначний underfit - RMSE валідаційного набора нижче, на поліномі 2 ступеня найкращий результат - RMSE приблизно однакові, а ось далі зі збільшенням ступеня полінома починається overfit. 

**Завдання 5**. Натренуйте моделі Lasso(), Ridge(), ElasaticNet() на цих даних (з поліном ознаками до степені 20 включно), порівняйте якість з тою, яка була отримана з лінійною регресією. Яка модель найкраще генералізує і чому на ваш погляд (можливо треба буде для відповіді зробити додатковий аналіз ознак)?

In [51]:
models = [
    LinearRegression(),
    Ridge(),
    Lasso(),
    ElasticNet(),
    ElasticNet(alpha=0.5)
]

def evaluate_model(model, X_train, y_train, X_val, y_val):
    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_val)

    train_metrics = round(root_mean_squared_error(y_train_pred, y_train),2)
    val_metrics = round(root_mean_squared_error(y_val_pred, y_val),2)

    return dict(train=train_metrics, val=val_metrics)

In [53]:
import warnings

warnings.filterwarnings("ignore")

for init in range(2,15):
    print('degree=', init)
    poly_features = PolynomialFeatures(degree=init)
    X_train_poly = poly_features.fit_transform(X_train)
    X_val_poly = poly_features.transform(X_val)

    for model in models:
        model.fit(X_train_poly, y_train)
        eval_results  = evaluate_model(model, X_train_poly, y_train, X_val_poly, y_val)
        print(f'{str(model)}: {eval_results}\n')

degree= 2
LinearRegression(): {'train': 1.02, 'val': 1.01}

Ridge(): {'train': 1.17, 'val': 1.23}

Lasso(): {'train': 1.57, 'val': 1.51}

ElasticNet(): {'train': 17.39, 'val': 20.76}

ElasticNet(alpha=0.5): {'train': 10.98, 'val': 13.21}

degree= 3
LinearRegression(): {'train': 0.8, 'val': 1.92}

Ridge(): {'train': 1.81, 'val': 7.88}

Lasso(): {'train': 1.87, 'val': 1.94}

ElasticNet(): {'train': 12.41, 'val': 19.29}

ElasticNet(alpha=0.5): {'train': 9.91, 'val': 20.44}

degree= 4
LinearRegression(): {'train': 0.0, 'val': 19.27}

Ridge(): {'train': 1.54, 'val': 27.1}

Lasso(): {'train': 1.88, 'val': 2.02}

ElasticNet(): {'train': 11.61, 'val': 17.54}

ElasticNet(alpha=0.5): {'train': 9.15, 'val': 15.88}

degree= 5
LinearRegression(): {'train': 0.0, 'val': 29.43}

Ridge(): {'train': 1.26, 'val': 29.49}

Lasso(): {'train': 1.95, 'val': 2.37}

ElasticNet(): {'train': 11.66, 'val': 43.16}

ElasticNet(alpha=0.5): {'train': 8.59, 'val': 34.96}

degree= 6
LinearRegression(): {'train': 0.0, 'v

Найкраще всіх генералізує лінійна регресія з поліномом 2 ступеню. Модель проста, але в поліномі 21 ознака, тому є сенс проаналізувати їх.